In [1]:
import pandas as pd
import numpy as np

# Reading dataset in
df = pd.read_csv("../data/raw/final_shop_6modata.csv")
df.head()

,Ad Group,Month,Impressions,Clicks,CTR,Conversions,Conv Rate,Cost,CPC,Revenue,Sale Amount,P&L
0,Shop - 1:1 - Desk - [shop coupon code],July,16038,6504,0.41,1166,0.10,6669,1.03,6402,136770.05,-267.086
1,Shop - 1:1 - Desk - [shop coupon],July,36462,14367,0.39,2188,0.09,13746,0.96,13262,283215.21,-483.951
2,Shop - 1:1 - Desk - [shop discount code],July,3635,1458,0.40,248,0.09,1606,1.10,1723,39165.46,117.136
3,Shop - 1:1 - Desk - [shop promo code],July,26185,10418,0.40,2294,0.12,13278,1.27,13042,284823.48,-235.921
4,Shop - 1:1 - Desk - [shop promo],July,808,282,0.35,61,0.15,391,1.39,337,7717.77,-53.604


In [2]:
# Count distinct Ad Group values
print("Unique Ad Groups: ",df["Ad Group"].nunique(), "\n")
print(df["Ad Group"].unique())

Unique Ad Groups:  40 

['Shop - 1:1 - Desk - [shop coupon code]'
 'Shop - 1:1 - Desk - [shop coupon]'
 'Shop - 1:1 - Desk - [shop discount code]'
 'Shop - 1:1 - Desk - [shop promo code]'
 'Shop - 1:1 - Desk - [shop promo]'
 'Shop - 1:1 - Mob - [shop coupon code]'
 'Shop - 1:1 - Mob - [shop coupon]'
 'Shop - 1:1 - Mob - [shop discount code]'
 'Shop - 1:1 - Mob - [shop promo code]' 'Shop - 1:1 - Mob - [shop promo]'
 'Shop - Exact - Desk - Competitor' 'Shop - Exact - Desk - Coupon Code'
 'Shop - Exact - Desk - Discount Code'
 'Shop - Exact - Desk - Free Shipping' 'Shop - Exact - Desk - Offer'
 'Shop - Exact - Desk - Promo Code' 'Shop - Exact - Desk - Sale'
 'Shop - Exact - Mob - Competitor' 'Shop - Exact - Mob - Coupon Code'
 'Shop - Exact - Mob - Discount Code' 'Shop - Exact - Mob - Free Shipping'
 'Shop - Exact - Mob - Offer' 'Shop - Exact - Mob - Promo Code'
 'Shop - Exact - Mob - Sale' 'Shop - Phrase - Desk - Competitor'
 'Shop - Phrase - Desk - Coupon Code'
 'Shop - Phrase - Desk - 

In [3]:
# MODIFY / CLEAN DATA

# Rename column for clarity
df.rename(columns= {"Sale Amount": "Quantity Sold"}, inplace = True)

# Ad Group column -- removes repeating "Shop - " value in every row
df["Ad Group"] = df["Ad Group"].str.slice(7)

df.head()

,Ad Group,Month,Impressions,Clicks,CTR,Conversions,Conv Rate,Cost,CPC,Revenue,Quantity Sold,P&L
0,1:1 - Desk - [shop coupon code],July,16038,6504,0.41,1166,0.10,6669,1.03,6402,136770.05,-267.086
1,1:1 - Desk - [shop coupon],July,36462,14367,0.39,2188,0.09,13746,0.96,13262,283215.21,-483.951
2,1:1 - Desk - [shop discount code],July,3635,1458,0.40,248,0.09,1606,1.10,1723,39165.46,117.136
3,1:1 - Desk - [shop promo code],July,26185,10418,0.40,2294,0.12,13278,1.27,13042,284823.48,-235.921
4,1:1 - Desk - [shop promo],July,808,282,0.35,61,0.15,391,1.39,337,7717.77,-53.604


In [4]:
# Split Ad Groups into 3 parts: Structure, Device, Theme
df[["Keyword Match", "Device", "Theme"]] = df["Ad Group"].str.split(" - ", n=2, expand=True)

# Rename 'Ad Group' col 
df.rename(columns= {"Ad Group": "Ad Group Detail"}, inplace = True)

# clean Theme column -- remove square brackets if present
df["Theme"] = df["Theme"].str.strip("[]")

df.head()

,Ad Group Detail,Month,Impressions,Clicks,CTR,Conversions,Conv Rate,Cost,CPC,Revenue,Quantity Sold,P&L,Keyword Match,Device,Theme
0,1:1 - Desk - [shop coupon code],July,16038,6504,0.41,1166,0.10,6669,1.03,6402,136770.05,-267.086,1:1,Desk,shop coupon code
1,1:1 - Desk - [shop coupon],July,36462,14367,0.39,2188,0.09,13746,0.96,13262,283215.21,-483.951,1:1,Desk,shop coupon
2,1:1 - Desk - [shop discount code],July,3635,1458,0.40,248,0.09,1606,1.10,1723,39165.46,117.136,1:1,Desk,shop discount code
3,1:1 - Desk - [shop promo code],July,26185,10418,0.40,2294,0.12,13278,1.27,13042,284823.48,-235.921,1:1,Desk,shop promo code
4,1:1 - Desk - [shop promo],July,808,282,0.35,61,0.15,391,1.39,337,7717.77,-53.604,1:1,Desk,shop promo


In [5]:
# work on Theme column further
print(df["Theme"].unique())

['shop coupon code' 'shop coupon' 'shop discount code' 'shop promo code'
 'shop promo' 'Competitor' 'Coupon Code' 'Discount Code' 'Free Shipping'
 'Offer' 'Promo Code' 'Sale' 'Black Friday/Cyber Monday']


In [6]:
# Further group Theme column on similar categories

# Treat all below categories as one new value: "Coupon/Discount"
coupon_like = [
    "shop coupon code", "shop coupon", "shop discount code",
    "shop promo code", "shop promo",
    "Coupon Code", "Discount Code", "Promo Code", "Offer",
]

# Overwrite Theme in-place
df["Theme"] = np.where(df["Theme"].isin(coupon_like),
                       "Coupon/Discount",
                       df["Theme"])

# List new values in Theme column
print("New 'Theme' column unique values: \n", df["Theme"].unique())

New 'Theme' column unique values: 
 ['Coupon/Discount' 'Competitor' 'Free Shipping' 'Sale'
 'Black Friday/Cyber Monday']


In [7]:
# Expand "Device" values for clarity
device_map = {
    "Desk": "Desktop",
    "Mob": "Mobile"
}

df["Device"] = df["Device"].replace(device_map)
df.head()

,Ad Group Detail,Month,Impressions,Clicks,CTR,Conversions,Conv Rate,Cost,CPC,Revenue,Quantity Sold,P&L,Keyword Match,Device,Theme
0,1:1 - Desk - [shop coupon code],July,16038,6504,0.41,1166,0.10,6669,1.03,6402,136770.05,-267.086,1:1,Desktop,Coupon/Discount
1,1:1 - Desk - [shop coupon],July,36462,14367,0.39,2188,0.09,13746,0.96,13262,283215.21,-483.951,1:1,Desktop,Coupon/Discount
2,1:1 - Desk - [shop discount code],July,3635,1458,0.40,248,0.09,1606,1.10,1723,39165.46,117.136,1:1,Desktop,Coupon/Discount
3,1:1 - Desk - [shop promo code],July,26185,10418,0.40,2294,0.12,13278,1.27,13042,284823.48,-235.921,1:1,Desktop,Coupon/Discount
4,1:1 - Desk - [shop promo],July,808,282,0.35,61,0.15,391,1.39,337,7717.77,-53.604,1:1,Desktop,Coupon/Discount


In [11]:
from pathlib import Path
# Make sure processed folder exists
Path("../data/processed").mkdir(parents=True, exist_ok=True)

# Save cleaned data
df.to_parquet("../data/processed/ads_clean.parquet", index=False)
df.to_csv("../data/processed/test_ads_clean.csv", index=False)

DATA CLEANING ENDS HERE, below is exploratory

In [27]:
# Aggregate by Ad Group
adgroup_perf = (
    df.groupby("Ad Group Detail", as_index=False)
      .agg({
          "P&L": "sum",          
          "Impressions": "sum",   
          "Clicks": "sum",
          "Conversions": "sum",
          "Revenue": "sum", 
          "Cost": "sum",
          "Quantity Sold": "sum"   
      })
)

In [28]:
# Rank by P&L, descending
adgroup_pnl_ranked = adgroup_perf.sort_values("P&L", ascending=False)

# Display top 10 Ad Groups by P&L
adgroup_pnl_ranked.style.format({"P&L": "{:.0f}"}) #rounding to nearest integer value for display clarity
adgroup_pnl_ranked.head(10) 

,Ad Group Detail,P&L,Impressions,Clicks,Conversions,Revenue,Cost,Quantity Sold
18,Exact - Mob - Black Friday/Cyber Monday,115.963,3662,266,24,160,44,3268.63
33,Phrase - Mob - Competitor,45.823,330,77,10,87,41,1736.24
10,Exact - Desk - Black Friday/Cyber Monday,41.946,257,24,7,45,3,898.80
29,Phrase - Desk - Free Shipping,11.538,342,38,3,28,17,841.93
26,Phrase - Desk - Competitor,6.070,109,10,2,13,7,261.40
14,Exact - Desk - Free Shipping,-15.365,4095,900,139,346,362,8960.61
36,Phrase - Mob - Free Shipping,-18.450,636,109,5,11,30,230.95
16,Exact - Desk - Promo Code,-24.364,51253,16814,3592,23821,23846,526442.64
23,Exact - Mob - Offer,-74.729,11549,2542,195,1484,1559,31302.47
11,Exact - Desk - Competitor,-121.942,3969,905,119,810,932,16489.23


In [29]:
import plotly.express as px

# Aggregate across all Ad Groups by month
month_perf = (
    df.groupby("Month", as_index=False)
      .agg({
          "Conversions": "sum",
          "Cost": "sum"
      })
)

# Put months in calendar order
month_order = ["July", "August", "September", "October", "November", "December"]
month_perf["Month"] = pd.Categorical(month_perf["Month"],
                                     categories=month_order,
                                     ordered=True)
month_perf = month_perf.sort_values("Month")


# Conversions per unit cost (quantity sold / cost)
month_perf["conv_per_cost"] = month_perf["Conversions"] / month_perf["Cost"]

In [31]:
# Create new metric, conversions per impression (to rank most effective ad group)
adgroup_perf["conv_per_impr"] = adgroup_perf["Conversions"] / adgroup_perf["Impressions"]

top3_conv_rate = (
    adgroup_perf
    .sort_values("conv_per_impr", ascending=False)
    .head(3)
)

top3_conv_rate[["Ad Group Detail", "Impressions", "Clicks", "Conversions", "conv_per_impr"]]


,Ad Group Detail,Impressions,Clicks,Conversions,conv_per_impr
3,1:1 - Desk - [shop promo code],142526,54312,12439,0.087275
16,Exact - Desk - Promo Code,51253,16814,3592,0.070084
0,1:1 - Desk - [shop coupon code],87936,33805,6094,0.069300


In [32]:
funnel_df = top3_conv_rate.melt(
    id_vars="Ad Group Detail",
    value_vars=["Impressions", "Clicks", "Conversions"],
    var_name="Stage",
    value_name="Count"
)

fig = px.funnel(
    funnel_df,
    x="Count",
    y="Stage",
    color="Ad Group Detail",
    title="Impressions → Clicks → Conversions Funnel (Top 3 Ad Groups by Conversions/Impressions)",
)

fig.show()

In [33]:
fig = px.line(
    month_perf,
    x="Month",
    y="conv_per_cost",
    markers=True,
    title="Conversions $1 by Month (All Ad Groups)",
    labels={"conv_per_cost": "Conversions per $1"}
)

fig.show()